In [14]:
# !pip install sentence_transformers openai
# !pip install pinecone-client
# !pip install docx2txt
# !pip install pipeline

# # !pip3 install git+https://git@github.com/pinecone-io/pinecone-python-client.git

In [2]:
import os
os.environ['OPENAI_API_KEY'] = ""

In [3]:
#This is for embedding. In here, one LM model from huggingface used.

from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')


text ='Abc'
# model.encode(text).tolist() #exmple how to do encoding.

In [4]:
#Function to split long documents in to smaller parts
def split_text_into_chunks(plain_text, max_chars=2000):
    """convert text into chunks"""
    
    text_chunks = []
    current_chunk = ""
    for line in plain_text.split("\n"):
        if len(current_chunk) + len(line) + 1 <= max_chars:
            current_chunk += line + " "
        else:
            text_chunks.append(current_chunk.strip())
            current_chunk = line + " "
    if current_chunk:
        text_chunks.append(current_chunk.strip())
    return text_chunks

In [5]:
import pinecone
pinecone.init(api_key="dae36959-08bc-4f9e-8468-3741321386b6", environment="us-east-1-aws") #Todo: Initialization of vector database module
index = pinecone.Index("squad") #Todo: Fill out with index name.


In [6]:

# import pinecone from sentence_transformers
# import SentenceTransformer from transformers
# import pipeline

# pinecone.init(api_key="dae36959-08bc-4f9e-8468-3741321386b6", environment="us-east-1-aws")
# index = pinecone.Index("squad")

# # Retrievers embed the context passages and questions
# retriever = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
# # Readers extract the answer from the returned context
# reader = pipeline(
#   tokenizer='deepset/electra-base-squad2',
#   model='deepset/electra-base-squad2',
#   task='question-answering'
# )
# question = "How much oil does Egypt produce in a day?"
# embed = retriever.encode([question]).tolist()
# # search for context passage with the answer
# matches = index.query(embed, top_k=5, include_metadata=True)["matches"]
# # extract the answer from the context passages
# for m in matches:
#   answer = reader(question=question, context=m["metadata"]["context"])
#   print(answer)

In [7]:
def addData(corpusData):
    """encode all the splitted chunks and add them to the vector database"""
    
    id  = index.describe_index_stats()['total_vector_count']
    for i in range(len(corpusData)):
        chunk=corpusData[i]
        chunkInfo=(str(id+i),
                model.encode(chunk).tolist(), #We are using the model to encode the original chunk of text.
                {'context': chunk}) #In metadata we are storing the original text here as context. 
        index.upsert(vectors=[chunkInfo])

In [8]:
import docx2txt

docx_file = "./DataLaw.docx"

def index_populating(docx_file):
    """read docx file and call split_text_into_chunks function to split it into chunks then 
    call addData() to add it to the index"""
    
    # extract text from docx file
    plain_text = docx2txt.process(docx_file)
    # # extract text and write images in /tmp/img_dir
    # text = docx2txt.process("file.docx", "/tmp/img_dir") 
    
    # split plain_text into chunks
    corpusData = split_text_into_chunks(plain_text)
    # encode chuncks and index it
    addData(corpusData)
    
index_populating(docx_file)

In [ ]:
#This function is responsible for matching the input string with already existing data on vector database.

def find_match(query,k):
    """This function is responsible for matching the input string
    with already existing data on vector database"""

    # encode the query
    query_em = model.encode(query).tolist()
    result = index.query(query_em, top_k=k, includeMetadata=True)
    
    return [result['matches'][i]['metadata']['context'] for i in range(k)]

context = find_match(query,k)

In [12]:
from pprint import pprint
from transformers import pipeline

model_name = "gpt3"
# load the reader model into a question-answering pipeline
reader = pipeline(tokenizer=model_name, model=model_name, task="question-answering")

def create_prompt(context,query):
    #Todo: Should be generated with the context/contexts we find by doing semantaic search]
    """ extracts answer from the context passage"""
    
    results = []
    for c in context:
        # feed the reader the question and contexts to extract answers
        answer = reader(question=query, context=c)
        # add the context to answer dict for printing both together
        answer["context"] = c
        results.append(answer)
    # sort the result based on the score from reader model
    sorted_result = pprint(sorted(results, key=lambda x: x["score"], reverse=True))
    return sorted_result

prompt = create_prompt(context,query)

SyntaxError: invalid syntax (Temp/ipykernel_6428/909643776.py, line 2)

In [15]:
def generate_answer(prompt):
    #Todo: Pass the generated prompt and pass it to gpt-3 to get answers.
    

In [ ]:
def user_query(query):
    #Todo: Make all the things together.
    context = find_match(query,k)
    prompt = create_prompt(context,query)
    answer = generate_answer(prompt)
    return answer

In [ ]:
user_query("How can I do this?")